# 简单的接口应该接受函数，而不是类的实例

Python允许挂钩（hook）函数，通过该函数回调函数内的代码。例如，list类型的sort方法接受可选的key参数，用以指定每个索引位置上的值之间应该如何排序。

In [1]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


**示例1：**在字典里找不到待查询的键时打印一条信息，并返回0，以作为该键所对应的值

In [2]:
def log_missing():
    print('Key added')
    return 0

在字典里放入一系列键值对，并给出某些值的增量，然后就可以两次触发log_missing函数，并在控制台里打印两次消息。

In [3]:
from collections import defaultdict
current = {'green':12, 'blue':3}
increments = [('red', 5), ('blue', 17), ('orange',  9)]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


**示例2：**给defaultdict传入一个产生默认值的挂钩，并令其统计出该字典一共遇到了多少个缺失的键。

**实现1：**使用带状态的闭包。

In [4]:
def increment_with_report(current, increments):
    added_count = 0

    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0

    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount

    return result, added_count

In [5]:
result, count = increment_with_report(current, increments)
assert count == 2

**缺点：**读起来要比无状态的函数难懂一些。

**实现2：**定义一个小型的类，把需要追踪的状态封装起来。

In [6]:
class CountMissing(object):
    def __init__(self):
        self.added = 0

    def missing(self):
        self.added += 1
        return 0

In [7]:
counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

**缺点：**单看该类，依然不太容易理解CountMissing的意图，需要等看过了defaultdict的用法之后，才能明白。

**实现3：**在代码中定义__call__的特殊方法，使相关对象能够像函数那样得到调用。如果把这样的实例传给内置的callable函数，callable函数会返回True。

In [8]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0

In [9]:
counter = BetterCountMissing()
counter()
assert callable(counter)

In [10]:
counter = BetterCountMissing()
result = defaultdict(counter, current)  # Relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 2
print(result)

defaultdict(<__main__.BetterCountMissing object at 0x0000000016797080>, {'green': 12, 'blue': 20, 'red': 5, 'orange': 9})
